In [1]:
# ============================
#        IMPORTS
# ============================
import re
import statistics
import pandas as pd


In [2]:
# ============================
#     HELPER FUNCTIONS
# ============================
def count_tokens(line, delimiter):
    tokens = re.split(delimiter, line.strip())
    return len([t for t in tokens if t])

def is_numeric(token):
    try:
        float(token)
        return True
    except ValueError:
        pass
    if "�" in token:
        parts = [p.strip() for p in token.split("�") if p.strip()]
        if len(parts) == 2:
            return all(is_numeric(p) for p in parts)
        elif len(parts) == 1:
            return is_numeric(parts[0])
    return False

def numeric_ratio(line, delimiter):
    tokens = [t for t in re.split(delimiter, line.strip()) if t]
    if not tokens:
        return 0.0
    return sum(is_numeric(t) for t in tokens) / len(tokens)

def most_common(lst):
    return max(set(lst), key=lst.count) if lst else None

def safe_mean(x):
    return statistics.mean(x) if len(x) > 0 else 0

def safe_var(x):
    return statistics.variance(x) if len(x) > 1 else 0

def safe_cv(x):
    if len(x) == 0:
        return 0
    mean_val = safe_mean(x)
    if mean_val == 0:
        return 0
    return safe_var(x) ** 0.5 / mean_val

def compute_interval_overlap(interval1, interval2):
    start1, end1 = interval1
    start2, end2 = interval2
    return max(0, min(end1, end2) - max(start1, start2))

def intervals_overlap(interval1, interval2):
    return max(interval1[0], interval2[0]) < min(interval1[1], interval2[1])

def generate_row_pattern(tokens):
    return ''.join('N' if is_numeric(tok) else 'S' for tok in tokens)

def get_token_intervals_multi(line):
    tokens = []
    token_counts = {}
    parts = re.split(r'(\s{2,})', line)
    pos = 0
    for part in parts:
        if re.fullmatch(r'\s{2,}', part):
            pos += len(part)
        elif part:
            start = pos + 1
            end = pos + len(part)
            base = part.strip()
            token_counts[base] = token_counts.get(base, 0) + 1
            tokens.append({
                "key": f"{base} {token_counts[base]}" if token_counts[base] > 1 else base,
                "display": base,
                "interval": (start, end)
            })
            pos += len(part)
    return tokens

In [23]:
# ============================
#        CORE FUNCTIONS
# ============================
def segregate_blocks(lines, start_idx, end_idx):
    blocks = []
    block = None
    block_idx = 0

    for i in range(start_idx, end_idx):
        line = lines[i]
        if line.strip():
            if block is None:
                block = {
                    "idx": block_idx,
                    "start": i,
                    "end": None,
                    "lines": [],
                    "block_type": None,
                    "delimiter": None,
                    "headers": [],
                    "header_extent": 0,
                    "title": None,
                    "df": None,
                    "used_as_header_for": [],
                    "stats": {},
                    "row_patterns": [],
                    "modal_token_count": 0,
                    "token_count_cv": 0.0
                }
            block["lines"].append({
                "line_idx": i,
                "text": line,
                "count_multispace_tokens": count_tokens(line, r"\s{2,}"),
                "count_single_tokens": count_tokens(line, r"\s+"),
                "count_tab_tokens": count_tokens(line, r"\t"),
                "numeric_multispace_ratio": numeric_ratio(line, r"\s{2,}"),
                "numeric_single_ratio": numeric_ratio(line, r"\s+"),
                "numeric_tab_ratio": numeric_ratio(line, r"\t"),
                "line_len": len(line)
            })
        elif block is not None:
            block["end"] = i - 1
            blocks.append(block)
            block_idx += 1
            block = None

    if block is not None:
        block["end"] = end_idx - 1
        blocks.append(block)

    return blocks

# def compute_statistics(block):
    # get = lambda attr: [line[attr] for line in block["lines"]]
    # s, m, t = get("count_single_tokens"), get("count_multispace_tokens"), get("count_tab_tokens")
    # ns, nm, nt = get("numeric_single_ratio"), get("numeric_multispace_ratio"), get("numeric_tab_ratio")
    # lens = get("line_len")
    # block["stats"] = {
    #     "mean_single_tokens": statistics.mean(s), "var_single_tokens": statistics.variance(s), "cv_single_tokens": safe_cv(s),
    #     "mean_multispace_tokens": statistics.mean(m), "var_multispace_tokens": statistics.variance(m), "cv_multispace_tokens": safe_cv(m),
    #     "mean_tab_tokens": statistics.mean(t), "var_tab_tokens": statistics.variance(t), "cv_tab_tokens": safe_cv(t),
    #     "mean_line_len": statistics.mean(lens), "var_line_len": statistics.variance(lens), "cv_line_len": safe_cv(lens),
    #     "mean_numeric_single": statistics.mean(ns), "mean_numeric_multispace": statistics.mean(nm), "mean_numeric_tab": statistics.mean(nt)
    # }

def compute_statistics(block):
    s = []
    m = []
    t = []
    ns = []
    nm = []
    nt = []
    lens = []

    for line in block["lines"]:
        s.append(line["count_single_tokens"])
        m.append(line["count_multispace_tokens"])
        t.append(line["count_tab_tokens"])
        ns.append(line["numeric_single_ratio"])
        nm.append(line["numeric_multispace_ratio"])
        nt.append(line["numeric_tab_ratio"])
        lens.append(line["line_len"])


    block["stats"] = {
        "mean_single_tokens": safe_mean(s),
        "var_single_tokens": safe_var(s),
        "cv_single_tokens": safe_cv(s),

        "mean_multispace_tokens": safe_mean(m),
        "var_multispace_tokens": safe_var(m),
        "cv_multispace_tokens": safe_cv(m),

        "mean_tab_tokens": safe_mean(t),
        "var_tab_tokens": safe_var(t),
        "cv_tab_tokens": safe_cv(t),

        "mean_line_len": safe_mean(lens),
        "var_line_len": safe_var(lens),
        "cv_line_len": safe_cv(lens),

        "mean_numeric_single": safe_mean(ns),
        "mean_numeric_multispace": safe_mean(nm),
        "mean_numeric_tab": safe_mean(nt)
    }


def detect_header_extent(block, delimiter):
    patterns, title_line = [], None
    for i, line in enumerate(block["lines"]):
        tokens = [t for t in re.split(delimiter, line["text"].strip()) if t]
        pattern = generate_row_pattern(tokens)
        patterns.append(pattern)
        if i == 0 and pattern == "S":
            title_line = i
    start_i = title_line + 1 if title_line is not None else 0
    extent = 0
    for pattern in patterns[start_i:]:
        if all(c == "S" for c in pattern): extent += 1
        else: break
    return extent, title_line

def merge_headers_by_overlap(token_maps):
    base_row = token_maps[0]
    merged_headers = [{"name": tok["display"], "interval": tok["interval"]} for tok in base_row]
    for row in token_maps[1:]:
        for tok in row:
            matched = False
            for hdr in merged_headers:
                if intervals_overlap(hdr["interval"], tok["interval"]):
                    hdr["name"] += " " + tok["display"]
                    hdr["interval"] = (min(hdr["interval"][0], tok["interval"][0]), max(hdr["interval"][1], tok["interval"][1]))
                    matched = True
                    break
            if not matched:
                merged_headers.append({"name": tok["display"], "interval": tok["interval"]})
    return sorted(merged_headers, key=lambda x: x["interval"][0])

def extract_headers(block, delimiter):
    extent, title_line = detect_header_extent(block, delimiter)
    block["header_extent"] = extent
    block["title"] = block["lines"][title_line]["text"] if title_line is not None else None
    if extent == 0:
        return [], 0
    header_lines = block["lines"][title_line+1:title_line+1+extent] if title_line is not None else block["lines"][:extent]
    if extent == 1:
        token_objs = get_token_intervals_multi(header_lines[0]["text"])
        return [{"name": t["display"], "interval": t["interval"]} for t in token_objs], extent
    token_maps = [get_token_intervals_multi(line["text"]) for line in header_lines]
    return merge_headers_by_overlap(token_maps), extent

def generate_df(headers, header_extent, lines, delimiter):
    data_lines = lines[header_extent:]
    col_names = [h["name"] for h in headers]
    rows = []
    for line in data_lines:
        tokens = [t.strip() for t in re.split(delimiter, line["text"].strip()) if t.strip()]
        rows.append(tokens)
    return pd.DataFrame(rows, columns=col_names)

def assign_tokens_by_overlap(headers, lines, delimiter):
    n_cols, n_rows = len(headers), len(lines)
    matrix = [[None for _ in range(n_cols)] for _ in range(n_rows)]
    col_names = [h["name"] for h in headers]
    for i, line in enumerate(lines):
        tokens = get_token_intervals_multi(line["text"])
        for tok in tokens:
            best_match = max(
                range(n_cols),
                key=lambda j: compute_interval_overlap(tok["interval"], headers[j]["interval"]),
                default=None
            )
            if best_match is not None:
                matrix[i][best_match] = (matrix[i][best_match] or "") + (" " if matrix[i][best_match] else "") + tok["display"]
    return pd.DataFrame(matrix, columns=col_names)

def process_block(blocks, block_idx):
    block = blocks[block_idx]
    compute_statistics(block)
    lst = [l["count_multispace_tokens"] for l in block["lines"]]
    print(lst)
    mode_multi = most_common(lst)
    mean_numeric_single = block["stats"]["mean_numeric_single"]
    print(block_idx, mode_multi, mean_numeric_single)
    if mean_numeric_single < 0.3:
        if mode_multi > 1:
            if len(block["lines"]) < 6:
                headers, _ = extract_headers(block, r"\s{2,}")
                block["headers"], block["block_type"] = headers, "header-only"
                return
            block["block_type"] = "narrative"
            return
        elif mode_multi == 1:
            block["block_type"] = "narrative"
            return

    cv_multi = block["stats"]["cv_multispace_tokens"]
    cv_tab = block["stats"]["cv_tab_tokens"]
    mode_tab = most_common([l["count_tab_tokens"] for l in block["lines"]])

    delimiter = r"\t" if cv_tab == 0 and mode_tab > 1 else r"\s{2,}" if cv_multi == 0 and mode_multi > 1 else None
    if delimiter:
        block["delimiter"] = delimiter
        headers, extent = extract_headers(block, delimiter)
        block["headers"], block["header_extent"] = headers, extent
        if extent == 0:
            for prev in reversed(blocks[:block_idx]):
                if prev["block_type"] == "header-only":
                    prev_headers = prev["headers"]
                    prev_width = len(prev_headers)
                    data_mode = most_common([count_tokens(line["text"], delimiter) for line in block["lines"]])
                    df = generate_df(prev_headers, 0, block["lines"], delimiter) if data_mode == prev_width else assign_tokens_by_overlap(prev_headers, block["lines"], delimiter)
                    block["df"], block["block_type"] = df, "data"
                    prev["used_as_header_for"].append(block["idx"])
                    return
            block["block_type"] = "narrative"
            return
        block["df"] = generate_df(headers, extent, block["lines"], delimiter)
        block["block_type"] = "complete-tabular"
        return

    delimiter = r"\s{2,}"
    block["delimiter"] = delimiter
    headers, extent = extract_headers(block, delimiter)
    block["headers"], block["header_extent"] = headers, extent
    if extent > 0:
        data_lines = block["lines"][extent:]
        token_counts = [count_tokens(l["text"], delimiter) for l in data_lines]
        new_cv = safe_cv(token_counts)
        block["df"] = generate_df(headers, extent, block["lines"], delimiter) if new_cv == 0 else assign_tokens_by_overlap(headers, data_lines, delimiter)
        block["block_type"] = "complete-tabular"
    else:
        for prev in reversed(blocks[:block_idx]):
            if prev["block_type"] == "header-only":
                prev_headers = prev["headers"]
                prev_width = len(prev_headers)
                data_mode = most_common([count_tokens(l["text"], delimiter) for l in block["lines"]])
                df = generate_df(prev_headers, 0, block["lines"], delimiter) if data_mode == prev_width else assign_tokens_by_overlap(prev_headers, block["lines"], delimiter)
                block["df"], block["block_type"] = df, "data"
                prev["used_as_header_for"].append(block["idx"])
                return
        block["block_type"] = "narrative"

# ============================
#        ENTRY POINT
# ============================
def parse(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    lines = text.splitlines()
    i = next((j for j, line in enumerate(lines) if line.startswith("DATA:")), None)
    if i is None:
        raise ValueError("No 'DATA:' line found. Not a templated NOAA file.")
    blocks = segregate_blocks(lines, i + 1, len(lines))
    for idx in range(len(blocks)):
        process_block(blocks, idx)
    return blocks


In [ ]:
blocks = parse("test.txt")

for id, block in enumerate(blocks):
    print(id)
    print(block["title"])
    print(block["block_type"])
    display(block["df"])

[]
None
narrative


None

[{'name': 'Sample Number', 'interval': (1, 6)}, {'name': '238U (ppb)', 'interval': (15, 19)}, {'name': '232Th (ppt)', 'interval': (27, 31)}, {'name': '230Th/232Th (atomic x10-6)', 'interval': (36, 49)}, {'name': 'd234U* (measured)', 'interval': (53, 62)}, {'name': '230Th/238U (activity)', 'interval': (68, 77)}, {'name': '230Th Age (uncorrected)', 'interval': (82, 94)}, {'name': '230Th AgeBP (corrected)', 'interval': (96, 107)}, {'name': '230Th Age (AD) (corrected)', 'interval': (111, 124)}]
None
header-only


None

[]
None
data


...

,Depth(cm),YearAD,d18O (per mil PDB)
0,0.005,2005.670,-6.102
1,0.01,2004.366,-6.987
2,0.015,2003.062,-6.841
3,0.02,2001.758,-7.136
4,0.025,2000.454,-6.968
...,...,...,...
75,0.535,1907.872,-7.053
76,0.54,1906.568,-6.978
77,0.545,1905.264,-7.054
78,0.55,1903.960,None


[]
None
narrative


None

[{'name': 'Depth(cm)', 'interval': (1, 9)}, {'name': 'YearAD', 'interval': (13, 18)}, {'name': 'd18O (per mil PDB)', 'interval': (25, 42)}]
None
complete-tabular


,Depth(cm),YearAD,d18O (per mil PDB)
0,0.004,1907.462,-6.751
1,0.020,1905.214,-6.971
2,0.040,1902.404,-6.876
3,0.060,1899.594,-6.824
4,0.080,1896.784,-6.771
...,...,...,...
516,13.830,1094.966,-7.969
517,13.855,1093.372,-7.357
518,13.880,1091.778,-7.893
519,13.905,1090.184,-7.822


In [5]:
blocks = parse("test2.txt")

for block in blocks:
    print(block["headers"])
    print(block["title"])
    print(block["block_type"])
    display(block["df"])

[]
None
narrative


None

[{'name': 'Sample Number', 'interval': (1, 13)}, {'name': 'Depth to top (mm)', 'interval': (16, 27)}, {'name': '238U (ppb)', 'interval': (34, 38)}, {'name': '232Th (ppt)', 'interval': (51, 56)}, {'name': '230Th/232Th (atomic x 10-6)', 'interval': (69, 83)}, {'name': 'd234U* measured', 'interval': (94, 101)}, {'name': '230Th/238U activity', 'interval': (113, 122)}, {'name': 'Age (yr) uncorrected', 'interval': (133, 143)}, {'name': 'Age (yr) corrected', 'interval': (154, 162)}, {'name': 'd234U initial corrected', 'interval': (173, 185)}, {'name': 'Age (yr BP)* corrected', 'interval': (195, 207)}]
None
header-only


None

[]
Dongge Cave, China (D4)
data


...

,Depth,Age,d18O
0,Depth,Age,d18O
1,153.8,7678,-4.08
2,154.3,7680,-4.08
3,154.8,7682,-4.23
4,155.3,7684,-4.1
...,...,...,...
385,381.3,8552,-5.04
386,382.1,8555,-4.91
387,383,8559,-4.88
388,383.8,8562,-4.77


[{'name': 'Depth', 'interval': (1, 5)}, {'name': 'Age', 'interval': (16, 18)}, {'name': 'd18O', 'interval': (26, 29)}]
5. Paix�o Cave, Brazil (PX5)
complete-tabular


,Depth,Age,d18O
0,Depth,Age,d18O
1,90,8087,-5.07
2,90.5,8089,-5.37
3,91,8091,-5.49
4,91.5,8093,-5.57
5,92,8095,-4.79
6,92.5,8097,-4.17
7,93,8099,-4.86
8,93.5,8101,-4.51
9,94,8103,-4.16


[{'name': 'Depth', 'interval': (1, 5)}, {'name': 'Age', 'interval': (16, 18)}, {'name': 'd18O', 'interval': (26, 29)}]
6. Padre Cave, Brazil (PAD07)
complete-tabular


,Depth,Age,d18O
0,Depth,Age,d18O
1,199,7782,-6.151
2,200.5,7789,-6.372
3,202,7797,-6.474
4,203.5,7804,-6.387
...,...,...,...
195,407.5,8503,-6.19
196,409.8,8504,-6.21
197,412,8505,-6.43
198,416,8507,-6.45


In [6]:
blocks = parse("test3.txt")

for block in blocks:
    print(block["headers"])
    print(block["title"])
    print(block["block_type"])
    display(block["df"])

[]
None
narrative


None

[]
None
narrative


None

[{'name': 'Sample mid depth', 'interval': (1, 9)}, {'name': 'Sample length', 'interval': (13, 18)}, {'name': 'd18O air (per mil)', 'interval': (21, 29)}, {'name': 'Sample mid gas age (default)', 'interval': (33, 42)}, {'name': 'delta age (default)', 'interval': (45, 53)}, {'name': 'Sample mid gas age (LD min)', 'interval': (56, 65)}, {'name': 'delta age (LD min)', 'interval': (69, 77)}]
None
complete-tabular


...

None

[]
None
narrative


None

[{'name': 'Depth', 'interval': (1, 5)}, {'name': 'Age (Default)', 'interval': (13, 21)}, {'name': 'Age (LD min)', 'interval': (27, 34)}, {'name': 'd18O (per mil)', 'interval': (39, 47)}]
None
complete-tabular


,Depth,Age (Default),Age (LD min),d18O (per mil)
0,1105.282,8999.4,#N/A,-21.27
1,1105.382,9010.6,#N/A,-21.36
2,1105.482,9021.9,#N/A,-21.39
3,1105.582,9033.1,#N/A,-20.94
4,1105.682,9044.4,#N/A,-20.91
...,...,...,...,...
427,1133.587,#N/A,18956.4,-28.02
428,1133.596,#N/A,18968.7,-28.29
429,1133.604,#N/A,18981,-28.29
430,1133.613,#N/A,18993.4,-28.36


In [8]:
blocks = parse("test4.txt")

for block in blocks:
    print(block["headers"])
    print(block["title"])
    print(block["block_type"])
    display(block["df"])

[]
None
narrative


None

[]
None
narrative


None

[{'name': 'Depth', 'interval': (1, 5)}, {'name': 'Age*a', 'interval': (11, 15)}, {'name': 'TOC*b', 'interval': (19, 23)}, {'name': 'C:N', 'interval': (29, 31)}, {'name': 'd13C-TOC', 'interval': (35, 42)}, {'name': 'Depth', 'interval': (48, 52)}, {'name': 'Age', 'interval': (60, 62)}, {'name': 'TOC', 'interval': (69, 71)}, {'name': 'C:N', 'interval': (78, 80)}, {'name': 'd13C-TOC', 'interval': (85, 92)}]
None
complete-tabular


...

None

[{'name': 'Depth', 'interval': (1, 5)}, {'name': 'Age', 'interval': (10, 12)}, {'name': 'LSR', 'interval': (18, 20)}, {'name': 'DBD', 'interval': (28, 30)}, {'name': 'TOC-MAR', 'interval': (34, 40)}, {'name': 'Alk.', 'interval': (44, 47)}, {'name': 'SST', 'interval': (55, 57)}, {'name': 'Dino', 'interval': (63, 66)}, {'name': 'C28', 'interval': (73, 75)}, {'name': 'Phytol', 'interval': (79, 84)}, {'name': 'd13C37', 'interval': (88, 93)}, {'name': 'd13C38', 'interval': (97, 102)}, {'name': 'd13C28', 'interval': (106, 111)}, {'name': 'd13Cdin', 'interval': (116, 123)}]
None
complete-tabular


,Depth,Age,LSR,DBD,TOC-MAR,Alk.,SST,Dino,C28,Phytol,d13C37,d13C38,d13C28,d13Cdin
0,2.5 - - - -,0.2,10.7,0.38,74,None,None,1.84,4.14,4.27,-25.1,-24.8,None,None
1,7.5 - - -,0.7,10.7,0.38,74,1.52,19.1,1.74,3.94,3.87,None,-24.1,None,None
2,12.5 - - - -,1.2,10.7,0.40,79,None,None,1.90,3.29,3.56,-21.3,-22.7,None,None
3,17.5 - - - - - -,1.6,10.7,0.40,84,None,None,1.52,2.99,3.08,None,None,None,None
4,[11.5]*a 1.7 - - - -,None,10.7,0.40,86,None,None,3.53,5.16,2.05,None,None,-23.9,-23.2
5,22.5 - - - -,2.1,10.7,0.36,64,1.93,18.4,1.21,2.11,2.72,None,None,None,None
6,27.5 - - - - - -,2.6,10.7,0.36,81,None,None,1.54,2.79,3.42,None,None,None,None
7,32.5 - - - -,3.0,10.7,0.34,76,1.32,17.9,1.70,2.53,3.10,None,None,None,None
8,37.5 - - - - - -,3.5,10.7,0.34,74,None,None,1.54,2.58,3.02,None,None,None,None
9,42.5 - - - -,4.0,10.7,0.36,70,None,None,1.67,2.56,2.53,-23.5,-23.7,None,None


[]
None
narrative


None

In [25]:
blocks = parse("test5.txt")

for id, block in enumerate(blocks):
    # print(id, block)
    print(block["lines"])
    print(block["block_type"])
    # display(block["df"])

[1, 1, 1, 1, 1, 1, 2]
0 1 0.012987012987012988
[1]
1 1 0.0
[1, 5, 7]
2 1 0.0
[7, 7, 7, 7, 7, 7, 8, 7, 7]
3 7 0.5246913580246914
[1, 5, 7]
4 1 0.0
[7, 7, 7, 7, 7, 7, 7, 7, 7]
5 7 0.5308641975308642
[1, 5, 7]
6 1 0.0
[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
7 7 0.5357142857142857
[1, 5, 7]
8 1 0.0
[7, 7]
9 7 0.5555555555555556
[1, 1, 1, 1, 1, 1, 1, 1]
10 1 0.0125
[1, 1, 4, 6]
11 1 0.0
[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
12 6 0.7416666666666667
[{'line_idx': 57, 'text': 'Table A.', 'count_multispace_tokens': 1, 'count_single_tokens': 2, 'count_tab_tokens': 1, 'numeric_multispace_ratio': 0.0, 'numeric_single_ratio': 0.0, 'numeric_tab_ratio': 0.0, 'line_len': 8}, {'line_idx': 58, 'text': 'Results of Inter-depositional environment analysis.  ', 'count_multispace_tokens': 1, 'count_single_tokens': 5, 'count_tab_tokens': 1, 'numeric_multispace_ratio